In [1]:
import pandas as pd

In [7]:
df = pd.read_csv('Filtered_S&P_Stock_Data.csv')
df = df.set_index('Date')
df.head()

,Open,High,Low,Close,Adj Close,Volume,Filtered_Open
Date,,,,,,,
2000-01-03,1469.250000,1478.000000,1438.359985,1455.219971,1455.219971,931800000,1509.891113
2000-01-04,1455.219971,1455.219971,1397.430054,1399.420044,1399.420044,1009000000,1460.423078
2000-01-05,1399.420044,1413.270020,1377.680054,1402.109985,1402.109985,1085500000,1438.658516
2000-01-06,1402.109985,1411.900024,1392.099976,1403.449951,1403.449951,1092300000,1386.279016
2000-01-07,1403.449951,1441.469971,1400.729980,1441.469971,1441.469971,1225200000,1399.262688


In [8]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

In [10]:
df_train = df.set_index(df.index)[:'2019-05-16']
df_test = df.set_index(df.index)['2019-05-15':]

We will have to scale our value, which would require us to start by using zero-mean normalization on the denoised Open data

In [13]:
df_mean = df_train['Filtered_Open'].mean()

In [14]:
df_mean

1528.9837666632293

In [15]:
df_train['ZeroMeanFilteredOpen'] = df_mean
df_train.head()

,Open,High,Low,Close,Adj Close,Volume,Filtered_Open,ZeroMeanFilteredOpen
Date,,,,,,,,
2000-01-03,1469.250000,1478.000000,1438.359985,1455.219971,1455.219971,931800000,1509.891113,1528.983767
2000-01-04,1455.219971,1455.219971,1397.430054,1399.420044,1399.420044,1009000000,1460.423078,1528.983767
2000-01-05,1399.420044,1413.270020,1377.680054,1402.109985,1402.109985,1085500000,1438.658516,1528.983767
2000-01-06,1402.109985,1411.900024,1392.099976,1403.449951,1403.449951,1092300000,1386.279016,1528.983767
2000-01-07,1403.449951,1441.469971,1400.729980,1441.469971,1441.469971,1225200000,1399.262688,1528.983767


In [16]:
df_test['ZeroMeanFilteredOpen'] = df_mean
df_test.head()

,Open,High,Low,Close,Adj Close,Volume,Filtered_Open,ZeroMeanFilteredOpen
Date,,,,,,,,
2019-05-15,2820.379883,2858.679932,2815.080078,2850.959961,2850.959961,3125950000,2840.351719,1528.983767
2019-05-16,2855.800049,2892.149902,2855.800049,2876.320068,2876.320068,3338060000,2839.855921,1528.983767
2019-05-17,2858.600098,2885.479980,2854.229980,2859.530029,2859.530029,3257950000,2839.645614,1528.983767
2019-05-20,2841.939941,2853.860107,2831.290039,2840.229980,2840.229980,3288870000,2838.708026,1528.983767
2019-05-21,2854.020020,2868.879883,2854.020020,2864.360107,2864.360107,3218700000,2834.590994,1528.983767


Now, let's declare three variables containing a StandardScaler, without any additional parameters. 

We can't use the same scaler for the three Opens because we'll be using the scaler to un-scale predictions as well so the scaler properties must be specific to each Open data.

Then, we'll scale our column values and use that for training.

In [19]:
scaler = StandardScaler()
scaler.fit_transform(df_train['Open'])

ValueError: Expected 2D array, got 1D array instead:
array=[1469.25       1455.2199707  1399.42004395 ... 2820.12011719 2820.37988281
 2855.80004883].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.